In [8]:
import os
import enchant
from guess_language import guess_language
import re
import pickle
from nltk import sent_tokenize, word_tokenize
import pandas as pd

In [6]:
files=os.listdir('Lang_Eng/')

In [9]:
def clean_datafile(files):
    raw_text=open('Lang_Eng/'+files,'r')
    
    #Remove UTF-8 Char
    messy_list=[]
    for item in raw_text:
        messy_list.append(re.sub(r'(\\+([A-z]|[0-9]){3})','',item))
    
    #Prep the sentences for tokenization
    Eng_sent_prep=[]
    for item in messy_list:
        Eng_sent_prep.append((re.sub(r'(.!?)([A-Z]+)', r'\1 \2', item)))
    
    #Sentence tokenization
    Eng_sent=[]
    for line in Eng_sent_prep:
        sent_tokenize_list = sent_tokenize(line)
        Eng_sent.append(sent_tokenize_list)
    flatten_sent = [val for sublist in Eng_sent for val in sublist]
    
    #Remove non-english sentences
    Eng_only=[]
    for sent in flatten_sent:
        if guess_language(sent) =='en':
            Eng_only.append(sent)
            
    #Create word tokenized list
    Eng_word_ml=[]
    for item in Eng_only:
        dsent=[]
        words= word_tokenize(item)
        for item in words:
            if item.isalnum() == False:
                pass
            else:
                dsent.append(item.lower())
        if len(dsent)>1:
            Eng_word_ml.append(dsent)
            
    #Remove random char
    Eng_word_ml_cleaned=[]
    for sent in Eng_word_ml:
        dsent=[]
        if len(sent)>2:
            for word in sent:
                if word in 'bcdfghjklmnopqrstvwxz':
                    pass
                else:
                    dsent.append(word)
            Eng_word_ml_cleaned.append(dsent)    
            
    return Eng_word_ml_cleaned        


In [11]:
total_words=[]
for file in files:
    words=[]
    words=clean_datafile(file)
    total_words.append(words)

flatten_words = [val for sublist in total_words for val in sublist]

In [13]:
len(flatten_words)

69728

In [14]:
def pd_frame_maker(sentlist, label):
    df=pd.DataFrame({'sent': sentlist, 'label': label})
    df['sentence']=df['sent'].apply(lambda x: " ".join(x))
    df['length']=df['sent'].apply(lambda x: len(x))
    df=df[df['length']>1]
    df['sentence']=df['sentence'].drop_duplicates()
    df['language']=df['sentence'].apply(lambda x: guess_language(str(x)))
    df=df[df['language']=='en']
    df = df.dropna()   
    df = df.ix[:,:4]
    return df 

In [15]:
df=pd_frame_maker(flatten_words,0)

In [16]:
df.to_pickle('data/lang8_eng.pkl')